<a href="https://colab.research.google.com/github/riccardomarin/prog_algo_2022/blob/main/02_graphs_antipatie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import networkx as nx
import numpy as np
import random

Mostriamo l'implementazione del nostro algoritmo per colorare il grafo delle antipatie.

Abbiamo visto che per ipotesi può esserci un solo arco problematico. Quindi condurremo la nostra DFS per bicolorare il grafo. Se dovessimo accorgerci che questo non è possibile, modificheremo il colore di un nodo, per tricolorare il grafo.

In [ ]:
# Algoritmo di colorazione per le antipatie
def colora(G):
  # == DFS ricorsiva ==
  def DFSr(x,G,colori,c):
    colori[x] = c
    nonlocal n_group
    for y in G[x]:
      if colori[int(y)]==-1:
        DFSr(y,G,colori,1-c)
        n_group = max(2, n_group)
      # Aggiungiamo il caso in cui la colorazione
      # di x abbia rotto la bicolorazione. In tal
      # caso, y era già colorato -> ecco l'arco problematico!
      if colori[x] == colori[int(y)]:
        colori[x] = 2
        n_group = 3
  # ====================

  # Inizializzazione il colore
  colore = [-1 for v in G]
  n_group = 1
  # Per tutti i nodi
  for i in range(len(colore)):
    # Se nodo non è ancora colorato
    if colore[i] == -1:
      DFSr(i, G, colore, 0)

  return colore, n_group

Ora testiamo il nostro algoritmo su un po' di istanze. Ci creiamo un codice che, data una lista di antipatie, ne genera il grafo associato.

In [ ]:
def grafo_antipatie(antipatie):
  d = []
  for i in range(k):
    # Consideriamo il caso in cui nella lista ci sia una persona
    # che ha espresso antipatia verso sé stessa come un errore da ignorare
    if antipatie[i] != -1 and antipatie[i] != i:
      d.append((i,antipatie[i])) 
      d.append((antipatie[i],i))
  return d

In [ ]:
# Consideriamo k studenti
k = 10

# Generiamo casualmente un vettore che possa prendere i valori
# tra -1 e k
antipatie = np.random.choice(range(-1,k),k)
d = grafo_antipatie(antipatie)
print(f"Le antipiatie dichiarate sono:{antipatie}")
print(f"Corrisondenti liste di adiacenza:{d}")

# Uso networkx per crare il grafo
G = nx.Graph()
G.add_nodes_from(range(0,k))  # Se non si aggiungono i nodi, networkx perderà
                              # i nodi senza archi incidenti (non sa che esistono)
G.add_edges_from(d)

# Lancio il mio algoritmo
colori,n_gruppi = colora(G)

# Stampo i risultati
print(f"Numero gruppi: {n_gruppi}")
print(f"Colori assegnati:{colori}")

# Disegno il risultato
nx.draw(G, with_labels=True, font_weight='bold', node_color=colori)

# Già che ci sono, faccio un check per vedere se il mio algoritmo funziona
for i in d:
  assert(colori[i[0]] != colori[i[1]])

Facciamo un test di correttezza più esaustivo: proviamo per un gran numero di grafi e controlliamo se per qualcuno di loro va in errore.

In [ ]:
from progressbar import progressbar

for k in progressbar(range(3,100)):
  for j in range(0,300):
    antipatie = np.random.choice(range(-1,k),k)
    d = grafo_antipatie(antipatie)

    G = nx.Graph()
    G.add_nodes_from(range(0,k))  
    G.add_edges_from(d)

    colori,_ = colora(G)

    for i in d:
      assert(colori[i[0]] != colori[i[1]])

Proviamo ora a valutare il tempo di questo algoritmo:

In [ ]:
from time import time               # Fornisce funzioni di timing
durate_random = []

# Proviamo i grafi con nodi tra 2 e 300
for k in progressbar(range(3,250)):
  durata = 0
  # Visto che proveremo esempi a caso, faremo la media su 100 istanze
  for _ in range(100):
    # Creo l'istanza
    antipatie = np.random.choice(range(-1,k),k)
    d = grafo_antipatie(antipatie)
    # == START ==         
    start = time()        # Conto il costo di creazione del grafo
    G = nx.Graph()        # perché fa parte della soluzione proposta.
    G.add_nodes_from(range(0,k))  
    G.add_edges_from(d)
    colori = colora(G)
    # == STOP ==
    durata = durata + (time() - start)

  # Salvo la media delle istanze 
  durate_random.append(durata/100)

In [ ]:
import matplotlib.pyplot as plt 
plt.plot(durate_random)

Proviamo a considerare un'altra famiglia di grafi possibili. Consideriamo grafi connessi con $|V|$ archi. Questi sono alberi (quindi con $|V|-1$ archi) a cui aggiungiamo un ulteriore arco che prima non era presente.

In [ ]:
k = 10

# Uso network per generare un albero random con k nodi
G = nx.random_tree(k)

# Estraggo la matrice di adiacenza associata
A = nx.to_numpy_matrix(G)

# Calcolo il grafo complemento
# NOTA: faccio in modo che la diagonale non sia considerata
A_c = ((A - 1) * -1) - np.diag(np.ones((len(A),)))

# Guardo gli archi che partecipano al complemento
missing_edges = np.where(A_c)

# Ne scelgo uno a caso
idx = random.choice(range(0, len(missing_edges[0])))

# Lo aggiungo al mio grafo iniziale
A[missing_edges[0][idx],missing_edges[1][idx]] = 1
A[missing_edges[1][idx],missing_edges[0][idx]] = 1

# Coloro il nuovo grafo
G = nx.Graph(A)
colori,_ = colora(G)

# Disegno il risultato
nx.draw(G, with_labels=True, font_weight='bold', node_color=colori)

# Già che ci sono, faccio un check per vedere se il mio algoritmo funziona
for i in range(k):
  for j in range(k):
    if A[i,j]==1:
      assert(colori[i] != colori[j])

Proviamo a effettuare il timing. Cosa ci aspettiamo?

In [ ]:
from time import time               # Fornisce funzioni di timing
durate_tree = []

# Proviamo i grafi con nodi tra 2 e 300
for k in progressbar(range(3,250)):
  durata = 0
  # Visto che proveremo esempi a caso, faremo la media su 100 istanze
  for _ in range(100):
    antipatie = np.random.choice(range(-1,k),k)
    d = grafo_antipatie(antipatie)

    # == START ==
    # Voglio misurare il timing della creazione dell'albero
    # ma non dell'operazione di inserire l'arco extra, che è un mio
    # artificio solo per creare un'istanza più interessante.
    # NOTA: Inserire l'arco lo faccio sulla matrice di adiacenza
    #       dove le operazioni facilmente costano O(N^2)

    start = time()
    G = nx.random_tree(k)
    durata = durata + (time() - start)
    # == STOP ==

    # Estraggo la matrice di adiacenza associata
    A = nx.to_numpy_matrix(G)

    # Calcolo il grafo complemento
    # NOTA: faccio in modo che la diagonale non sia considerata
    A_c = ((A - 1) * -1) - np.diag(np.ones((len(A),)))

    # Guardo gli archi che partecipano al complemento
    missing_edges = np.where(A_c)

    # Ne scelgo uno a caso
    idx = random.choice(range(0, len(missing_edges[0])))

    # Lo aggiungo al mio grafo iniziale
    A[missing_edges[0][idx],missing_edges[1][idx]] = 1
    A[missing_edges[1][idx],missing_edges[0][idx]] = 1

    # == START ==
    start = time()     
    colori = colora(G)
    # == STOP ==
    durata = durata + (time() - start)

  # Salvo la media delle istanze 
  durate_tree.append(durata/100)

In [ ]:
import matplotlib.pyplot as plt 
plt.plot(durate_tree)

Visualizziamo i due timings insieme

In [ ]:
plt.plot(durate_random,color='b')
plt.plot(durate_tree, color='r')